# Image Caption Generator

### Import Section

In [16]:
from os import listdir
import re
import pandas as pd
import numpy as np

#Pickle
from pickle import dump
from pickle import load

#Keras
#Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

#NTLK
import nltk
#Lemmatization
from nltk.stem import WordNetLemmatizer
#corpus-bleu evaluation metrics
from nltk.translate.bleu_score import corpus_bleu
from tensorflow.keras.layers import Input
from tqdm import tqdm

from PIL import Image


### Preprocess image

### extract_features

In [3]:
# extrat Features
def extract_features(directory):
    
    model = VGG16( include_top=False,input_tensor=Input(shape=(128, 128, 3)))
    
    #remove last layer
    #model.layers.pop()
    model = Model(inputs = model.inputs , outputs = model.layers[-1].output)
    
    #print(model.summary())
    
    features = dict()
    
    #for name in listdir(directory):
    dirs  = listdir(directory)
    count = 0
    for name in tqdm(dirs, desc='dirs'):
        pass
        
        # load and image
        filename = directory + '/' + name
        image = load_img(filename , target_size=(128 , 128))
        
        image = img_to_array(image)
        
        image = image.reshape((1 , image.shape[0] , image.shape[1] ,image.shape[2]))
        
        image = preprocess_input(image)
        
        
        feature = model.predict(image , verbose = 0)
        
        # get image id
        image_id = name.split(".")[0]
        
        # store features
        features[image_id] = feature
        
        #print(name)
        count = count + 1
        if count == 10000:
            break
        
    return features

In [20]:
%%time
#Train
#directory = '/content/val2017'
#Valid Path
directory = 'D:/SiDi/Project/Modulo III/dataset/train2017'
#directory = 'D:/SiDi/Project/Modulo III/dataset/val2017'

#features = extract_features(directory)
#print('extracted features :',len(features))

model = VGG16()
#model = VGG16( include_top=False,input_tensor=Input(shape=(224, 224, 3)))
    
#remove last layer
#model.layers.pop()
model = Model(inputs = model.inputs , outputs = model.layers[-2].output)

#print(model.summary())

features = dict()

#for name in listdir(directory):
dirs  = listdir(directory)
#Train Image Dataset Path
#dirs  = listdir(TRAIN_IMAGE_DATASET_PATH)
#dirs = sorted(dirs)[0:10000]
#dirs = sorted(dirs)
count = 0
for name in tqdm(dirs, desc='dirs'):
    #pass

    # load and image
    filename = directory + '/' + name
    #image = load_img(filename , target_size=(128 , 128))
    image = load_img(filename , target_size=(224 , 224))
    #print(type(image))
    #target_size=(128 , 128)
    image = load_img(filename)

    image = image.resize((224, 224), Image.ANTIALIAS)
    image = img_to_array(image)

    image = image.reshape((1 , image.shape[0] , image.shape[1] ,image.shape[2]))

    image = preprocess_input(image)


    feature = model.predict(image , verbose = 0)

    # get image id
    image_id = name.split(".")[0]

    # store features
    features[image_id] = feature
    #print(image_id)
    #print(name)
    
#dump(features , open('./val_features.pkl' , 'wb'))
dump(features , open('./train_features.pkl' , 'wb'))

dirs: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 118287/118287 [4:50:19<00:00,  6.79it/s]


CPU times: total: 4h 3min 54s
Wall time: 4h 52min 24s


In [5]:
dump(features , open('./train_features.pkl' , 'wb'))

### Create Vocabulary from Images

In [8]:
print("Done")

Done


In [32]:
dump(features , open('features.pkl','wb'))

### Load Decriptions


In [18]:
#lemma = WordNetLemmatizer()

def load_decriptions(doc):
    mapping = dict()
    
    for i in range(len(doc)):
        #image_id = doc['image_id'][i]
        image_id = doc['filename'][i]
        image_desc = doc['desc'][i]
        
        if image_id not in mapping:
            mapping[image_id] = list()
        
        mapping[image_id].append(image_desc)
        
    return mapping

def clean_text(desc):
    
    # clean punctuation
    desc = re.sub(r'[^\w\s]' ,'', desc)
    
    # tokenize the words
    desc = desc.split()
    
    # convert to lower case
    desc = [token.lower() for token in desc]
    
    # lemmatization
    #desc = [lemma.lemmatize(token) for token in desc]
    
    # remove numerical values
    desc = [token for token in desc if token.isalpha()]
    
    # join whole token
    desc = ' '.join(desc)
    
    return desc

In [21]:
# convert loaded descriptions into vocablury
def to_vocabluary(descriptions):
    all_desc = set()
    
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key]]
        
    return all_desc

def save_descriptions(descriptions , filename):
    lines = list()
    
    for key , desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key +' '+ desc)
            
    data = '\n'.join(lines)
    file = open(filename , 'w')
    file.write(data)
    file.close()              

### Load Descriptions

In [12]:
descriptions = pd.read_csv('DataframeFinal.csv')

In [13]:
descriptions.head()

,Unnamed: 0,caption,filename,rotulos,image_height,image_width
0,4,"Esta com o braco esquerdo erguido, segurando ...",i-00000001.jpg,train,960,960
1,11,Ao centro uma arvore de natal com decoracao d...,i-00000002.jpg,train,608,1080
2,14,A sexta pessoa usa uma blusa listrada em pret...,i-00000002.jpg,train,608,1080
3,19,"Ao centro, um cano grosso na vertical e um fi...",i-00000004.jpg,train,1081,1080
4,20,Entre os canos e a parede cresce uma planta d...,i-00000004.jpg,train,1081,1080


In [ ]:
nltk.download('pt_core_news_sm')

In [14]:
descriptions['desc'] = descriptions['caption'].apply(lambda x : clean_text(str(x)))

In [16]:
type(descriptions['desc'])

pandas.core.series.Series

In [17]:
descriptions['desc'][0]

'esta com o braco esquerdo erguido segurando um crucifixo preso por uma alca'

In [ ]:
descriptions[descriptions["image_id"] == "2258277193_586949ec62.jpg.1"]

### Remove Outlier Instance

In [ ]:
descriptions = pd.read_csv('results.csv' , sep=',')
descriptions.columns = ['image_id','no','desc']
#descriptions = descriptions.drop(descriptions[descriptions["image_id"] == "2258277193_586949ec62.jpg.1"].index, inplace=False).copy()
#descriptions = descriptions[descriptions["image_id"] != "2258277193_586949ec62.jpg.1"].copy()
descriptions[descriptions["image_id"] == "2258277193_586949ec62.jpg.1"].index
#print(f"ANTES = {descriptions.shape}")
descriptions = descriptions.drop(descriptions[descriptions["image_id"] == "2258277193_586949ec62.jpg.1"].index, axis=0, inplace=False).copy()
#print(f"DEPOIS {descriptions.shape}")

descriptions[descriptions["image_id"] == "2258277193_586949ec62.jpg.1"].index
#descriptions.iloc[[6729, 6730, 6731, 6732, 6733]]

descriptions.reset_index(inplace=True)

In [ ]:
descriptions[descriptions["image_id"] == "2258277193_586949ec62.jpg.1"].index
#descriptions.iloc[[6729, 6730, 6731, 6732, 6733]]

descriptions.reset_index(inplace=True)
#descriptions.iloc[[6729, 6730, 6731, 6732, 6733]]


In [19]:
desc_map = load_decriptions(descriptions) # image name with Its captions

In [22]:
vocabulary = to_vocabluary(desc_map)

In [23]:
len(vocabulary)

58595

In [24]:
save_descriptions(desc_map , 'descriptions.txt')

### Train Set

### Load Documento 
### Load Clean Descriptions

In [53]:
# loading the doc
def load_doc(filename):
    file = open(filename , 'r')
    text = file.read()
    file.close()
    return text

# this function is used for to get train image description from our dataset

def load_clean_descriptions(filename , dataset):
    doc = load_doc(filename)
    descriptions = dict()
    
    for line in doc.split('\n'):
        tokens = line.split()
        image_id , image_desc = tokens[0] , tokens[1:]
        #print(f"image_id = {image_id}" )
        if image_id in dataset:
            
            if image_id not in descriptions:
                descriptions[image_id] = list()
            
            # we add two tage at start and at end of the descitpion to identify to start and 
            # end of desc.
            desc = 'startseq '+ ' '.join(image_desc)+ ' endseq'
            descriptions[image_id].append(desc)
            
    return descriptions


# laod photo features
def load_photo_features(filename , dataset):
    all_features = load(open(filename,'rb'))
    #print(all_features)
    #features = {k+'.jpg' : all_features[k] for k in dataset}
    features = {k: all_features[k.split(".")[0] ] for k in dataset}
    return features

In [26]:
#train = set(descriptions['image_id'][:127125])
train = set(descriptions['filename'][:7999])
print('len of train image',len(train))

len of train image 5162


In [27]:
# traininset makinng and set startseq and endseq tag in descriptions
train_descriptions = load_clean_descriptions('descriptions.txt' , train) 

In [28]:
print('len of train descriptions' , len(train_descriptions))

len of train descriptions 5162


In [29]:
train = pd.DataFrame(train)
"""train2 = train[0].apply(lambda x : x.replace('.jpg.1' , '.jpg').replace('.jpg' , '')) # remove jpg sign
#train2 = train2.apply(lambda x : x.replace('.jpg' , '')) # remove jpg sign
train2 = train2.drop(train2[train2 == "2258277193_586949ec62"].index)
"""

'train2 = train[0].apply(lambda x : x.replace(\'.jpg.1\' , \'.jpg\').replace(\'.jpg\' , \'\')) # remove jpg sign\n#train2 = train2.apply(lambda x : x.replace(\'.jpg\' , \'\')) # remove jpg sign\ntrain2 = train2.drop(train2[train2 == "2258277193_586949ec62"].index)\n'

0       i-00002015.jpg
1       i-00006457.jpg
2       i-00001626.jpg
3       i-00006132.jpg
4       i-00007167.jpg
             ...      
5157    i-00007264.jpg
5158    i-00007097.jpg
5159    i-00003544.jpg
5160    i-00002344.jpg
5161    i-00002391.jpg
Name: 0, Length: 5162, dtype: object

In [ ]:
train2[train2 == "2258277193_586949ec62"].sum()

In [48]:
filename_train = train[0]

In [ ]:
len(train[0]), len(train2)


In [39]:
all_features = load(open('features.pkl','rb'))

In [54]:
# train features means feautures from training images
#train_features = load_photo_features('features.pkl' , train2)
train_features = load_photo_features('features.pkl' , filename_train)
print('photos train :',len(train_features))

photos train : 5162


In [55]:
# convert dictonary to lis descriptions

def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
        
    return all_desc

# fit tokenizer on descriptions
def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [56]:
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('vocab size' , vocab_size)

vocab size 7838


In [58]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [205]:
# calculate the length with most words
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max([len(line.split())for line in lines])
    
# create sequences of images,input sequences and output sequences
def create_sequences(tokenizer , max_length , desc_list , photo):
    X1 , X2 , y = list() , list() , list()
    
    for desc in desc_list:
        # convert words to number value
        seq = tokenizer.texts_to_sequences([desc])[0]
        
        for i in range(1, len(seq)):
            
            in_seq , output_seq = seq[:i] , seq[i]
            in_seq = pad_sequences([in_seq] , maxlen = max_length)[0]
            output_seq = to_categorical([output_seq] , num_classes = vocab_size)[0]
            #print(photo)
            X1.append(photo)
            X2.append(in_seq)
            y.append(output_seq)
            
    return np.asarray(X1) , np.asarray(X2) , np.asarray(y)

### VGG16 + LSTM DeepModel

In [199]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , Dense , LSTM , Embedding , Dropout
#from tensorflow.keras.layers.merge import add
from tensorflow.keras.layers import Add
from tensorflow.keras.callbacks import EarlyStopping

In [219]:
def define_Model(vocab_size , max_length):
    
    # feature extractor model
    inputs1 = Input(shape=(4096, ))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(512 , activation='relu')(fe1)
    fe3 = Dense(256 , activation = 'relu')(fe2)
    
    # sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size,512,mask_zero=True )(inputs2) # mask_zero = ignore padding
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(512 , return_sequences=True)(se2)
    se4 = Dropout(0.5)(se3)
    se5 = LSTM(256)(se4)
    
    
    #decoder Model
    decoder1 = Add()([fe3,se5])
    decoder2 = Dense(256 , activation='relu')(decoder1)
    decoder3 = Dense(512 , activation='relu')(decoder2)
    outputs = Dense(vocab_size , activation='softmax')(decoder3)
    
    # combine both image and text
    model = Model([inputs1 , inputs2] , outputs)
    model.compile(loss='categorical_crossentropy' , optimizer = 'adam')
    
    # summary
    #print(model.summary())
    
    return model

### Data Generator

In [223]:
# below code progressivly load the code in batches
def data_generator(descriptions , photos , tokenizer , max_length):
    while 1:
        print("Data Generator ...")
        for key , desc_list in descriptions.items():
            print(f"Photo Array Size = {photos[key].shape}")
            #print(photos[key])
            #print(photos[key][0])
            photo = photos[key][0][0][0]
            """photos3d = photos[key][0]
            temp = []
            for photos2d in photos3d:
                for photo1d in photos2d:
                    #print(f"Photo = {photo1d.shape}")
                    #break
                    temp.append(photo1d)
            photo = np.asarray(temp)
            print(f"Photo = {photo.shape}")
            #print(f"Temp = {len(temp)}")
            """
            in_img , in_seq , out_seq = create_sequences(tokenizer , max_length , desc_list , photo)
            print(f"Image Input = {in_img.shape}")
            print(f"Seq Input = {in_seq.shape}")
            print(f"Concate Input = {len([in_img , in_seq])}")
            """temp = [] 
            for i in range(0,in_img.shape[0]):
                temp.append(in_img[i][0][0])
            in_img = np.asarray(temp)"""
            #yield in_img , in_seq, out_seq
            #yield[[in_img , in_seq] , out_seq]
                    
            """in_img = in_img.reshape(in_img.shape[1],in_img.shape[0])"""
            yield [[ in_img , in_seq], out_seq]
            
            
model = define_Model(vocab_size , max_len)
epochs = 5
steps = len(train_descriptions)

for i in range(epochs):
    generator = data_generator(train_descriptions , train_features , tokenizer , max_len)
    
    model.fit(generator , epochs = 1 , steps_per_epoch = steps , verbose = 1)
    #model.fit()
    
    model.save('model_'+ str(i+1) + '.h5')

Data Generator ...
Photo Array Size = (1, 4, 4, 512)
Image Input = (14, 512)
Seq Input = (14, 64)
Concate Input = 2
Photo Array Size = (1, 4, 4, 512)
Image Input = (24, 512)
Seq Input = (24, 64)
Concate Input = 2


InvalidArgumentError: Graph execution error:

Detected at node 'model_92/dense_462/MatMul' defined at (most recent call last):
    File "C:\Users\rbsa2\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\rbsa2\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "D:\SiDi\Project\Modulo III\experiment\image-caption-generator\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "D:\SiDi\Project\Modulo III\experiment\image-caption-generator\lib\site-packages\traitlets\config\application.py", line 972, in launch_instance
      app.start()
    File "D:\SiDi\Project\Modulo III\experiment\image-caption-generator\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "D:\SiDi\Project\Modulo III\experiment\image-caption-generator\lib\site-p

<b> Vocabulary Size Info  </b>

In [ ]:
print('len :', len(train))
print('Descriptions :',len(train_descriptions))
print('photos train :',len(train_features))
print('Vocabulary size :',vocab_size)
max_len = max_length(train_descriptions)
print('Description max length :', max_len)

In [ ]:
from keras.models import load_model

<b> Training Process </b>

In [109]:
vocab_size,max_len

(7838, 64)

In [110]:
model = define_Model(vocab_size , max_len)
epochs = 5
steps = len(train_descriptions)

for i in range(epochs):
    generator = data_generator(train_descriptions , train_features , tokenizer , max_len)
    
    model.fit(generator , epochs = 1 , steps_per_epoch = steps , verbose = 1)
    #model.fit()
    
    model.save('model_'+ str(i+1) + '.h5')

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 64)]         0           []                               
                                                                                                  
 embedding_12 (Embedding)       (None, 64, 512)      4013056     ['input_27[0][0]']               
                                                                                                  
 input_26 (InputLayer)          [(None, 4096)]       0           []                               
                                                                                                  
 dropout_37 (Dropout)           (None, 64, 512)      0           ['embedding_12[0][0]']           
                                                                                           

ValueError: in user code:

    File "D:\SiDi\Project\Modulo III\experiment\image-caption-generator\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "D:\SiDi\Project\Modulo III\experiment\image-caption-generator\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\SiDi\Project\Modulo III\experiment\image-caption-generator\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "D:\SiDi\Project\Modulo III\experiment\image-caption-generator\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "D:\SiDi\Project\Modulo III\experiment\image-caption-generator\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compile

### Bleu Score For Model Evalution

In [ ]:
def word_for_id(integer , tokenizer):
    for word,index in tokenizer.word_index.items():
        if index == integer:
            return word

In [ ]:
def generate_desc(model , tokenizer , photo , max_length):
    
    input_text = 'startseq'
    
    for i in range(max_length):
        
        sequence = tokenizer.texts_to_sequences([input_text])[0]
        sequence = pad_sequences([sequence] , maxlen=max_length)
        
        # predict the next word
        next_word_id = model.predict([photo,sequence],verbose = 0)
        
        # get highest probality word from list of words
        next_word_id = np.argmax(next_word_id)
        
        # get word from id
        word = word_for_id(next_word_id , tokenizer)
        
        if word is None:
            break
            
        # update input text
        input_text += ' '+ word
        
        if word == 'endseq':
            break
            
    return input_text

In [ ]:
def evaluate_model(model , desciptions , photos , tokenizer , max_length):
    actual , predicted = list() , list()
    
    for key , desc_list in desciptions.items():
        generated_desc = generate_desc(model , tokenizer , photos[key] , max_length)
        
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(generated_desc.split())
    
    print('Bleu_Score -1 = %f'%corpus_bleu(actual , predicted , weights=(1,0,0,0)))
    
    print('Bleu_Score -2 = %f'%corpus_bleu(actual , predicted , weights=(0.5,0.5,0,0)))
    
    print('Bleu_Score -3 = %f'%corpus_bleu(actual , predicted , weights=(0.33,0.33,0.33,0))) 
    
    print('Bleu_Score -4 = %f'%corpus_bleu(actual , predicted , weights=(0.25,0.25,0.25,0.25)))

In [ ]:
127125+(5000*5) # 5000 images for testing each image has 5 captions so that is calculation

In [ ]:
158915-152125 # reaminng Images total 158915

2nd test Dataset 

In [ ]:
6790 / 5 # test Set Which test at the last

### Test Set

In [ ]:
descriptions = pd.read_csv('results.csv' , sep=',')
descriptions.columns = ['image_id','no','desc']
#descriptions = descriptions.drop(descriptions[descriptions["image_id"] == "2258277193_586949ec62.jpg.1"].index, inplace=False).copy()
#descriptions = descriptions[descriptions["image_id"] != "2258277193_586949ec62.jpg.1"].copy()
descriptions[descriptions["image_id"] == "2258277193_586949ec62.jpg.1"].index
#print(f"ANTES = {descriptions.shape}")
descriptions = descriptions.drop(descriptions[descriptions["image_id"] == "2258277193_586949ec62.jpg.1"].index, axis=0, inplace=False).copy()
#print(f"DEPOIS {descriptions.shape}")

descriptions[descriptions["image_id"] == "2258277193_586949ec62.jpg.1"].index
#descriptions.iloc[[6729, 6730, 6731, 6732, 6733]]

descriptions.reset_index(inplace=True)
test = set(descriptions['image_id'][20227:])
print('len of train image',len(test))

#test2 = test[0].apply(lambda x : x.replace('.jpg' , '')) # remove jpg sign
test_descriptions = load_clean_descriptions('descriptions.txt',test)
print('len of train image',len(test_descriptions))


In [ ]:
test = pd.DataFrame(test)
test2 = test[0].apply(lambda x : x.replace('.jpg.1' , '.jpg').replace('.jpg' , '')) # remove jpg sign
#train2 = train2.apply(lambda x : x.replace('.jpg' , '')) # remove jpg sign
test2 = test2.drop(test2[test2 == "2258277193_586949ec62"].index)


In [ ]:
test_features = load_photo_features('features.pkl',test2)

In [ ]:
from keras.models import load_model

In [ ]:
filename = "/content/model_5.h5"
model = load_model(filename)

In [ ]:
print('len of test dataset',len(test))
print('len of Descriptions',len(test_descriptions))
print('len of Test features',len(test_features))

### Evaluate Model Test Dataset

In [ ]:
evaluate_model(model , test_descriptions , test_features , tokenizer , max_len)

In [ ]:
dump(tokenizer , open('tokenizer.pkl','wb'))

### Testing on Individual Images

In [ ]:
def word_for_id(integer , tokenizer):
    for word,index in tokenizer.word_index.items():
        if index == integer:
            return word
        
def generate_desc(model , tokenizer , photo , max_length):
    
    input_text = 'startseq'
    
    for i in range(max_length):
        
        sequence = tokenizer.texts_to_sequences([input_text])[0]
        sequence = pad_sequences([sequence] , maxlen=max_length)
        
        # predict the next word
        next_word_id = model.predict([photo,sequence],verbose = 0)
        
        # get highest probality word from list of words
        next_word_id = np.argmax(next_word_id)
        
        # get word from id
        word = word_for_id(next_word_id , tokenizer)
        
        if word is None:
            break
            
        # update input text
        input_text += ' '+ word
        
        if word == 'endseq':
            break
            
    return input_text

In [ ]:
def extract_features_for_one_image(filename):
    model = VGG16()
    model.layers.pop()
    
    model = Model(inputs = model.inputs , outputs = model.layers[-1].output)
    
    image = load_img(filename , target_size=(224,224))
    
    image = img_to_array(image)
    
    image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
    
    image = preprocess_input(image)
    
    feature = model.predict(image , verbose = 0)
    
    return feature

In [ ]:
def get_image_caption(filename,model):
    tokenizer = load(open('tokenizer.pkl','rb'))
    photo = extract_features_for_one_image(filename)
    desc = generate_desc(model , tokenizer , photo , 80)
    desc = desc.replace('startseq','')
    desc = desc.replace('endseq','')
    return desc.title()

In [ ]:
model2 = load_model('model_1.h5')

In [ ]:
from IPython.display import Image,display

## 1.

In [ ]:
filename = 'test_image.jpg'
print(get_image_caption(filename,model2))
display(Image(filename))

## 2.

In [ ]:
filename = 'test_image2.jpg'
print(get_image_caption(filename,model2))
display(Image(filename))

In [ ]:
filename = '984950.jpg'
print(get_image_caption(filename,model2))
display(Image(filename))

In [ ]:
filename = 'MyImage.jpg'
print(get_image_caption(filename,model2))
display(Image(filename))